In [ ]:
import pandas as  pd
import numpy as np
import warnings
import itertools
import xgboost as xgb
import lightgbm as lgb
from tqdm import tqdm
from numpy import loadtxt
import time
import gc
warnings.filterwarnings('ignore')
kernel_with_output = True
np.random.seed(10)
%matplotlib inline

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', 231)
pd.set_option('display.max_columns', 100)

In [ ]:
Validation = False
start_time = time.time()

In [ ]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int16)
    return df

In [6]:
if kernel_with_output:
    item =pd.read_csv("../data/items.csv")
    item_cat = pd.read_csv("../data/item_categories.csv")
#     df_shops = pd.read_csv("../data/shops.csv")
    sale_train = pd.read_csv("../data/sales_train.csv.gz")
    test = pd.read_csv("../data/test.csv.gz")
    temp = pd.read_csv("../data/sample_submission.csv.gz")


In [7]:
train_test_set = pd.read_pickle("../results/07/all_data_traintest_012367.pkl")
train_test_set.shape

(8812244, 124)

In [8]:
train_test_set.head()
train_test_set.dtypes
train_test_set.last_sale_item.isna().sum()
train_test_set.last_sale_shop_item.isna().sum()


,Unnamed: 0,Unnamed: 0.1,date_block_num,item_category_id,item_cnt_month,item_id,item_price,shop_id,prev_shopitem_sales_1,prev_shopitem_sales_2,prev_shopitem_sales_3,prev_shopitem_sales_4,prev_shopitem_sales_5,prev_shopitem_sales_6,prev_shopitem_sales_7,prev_shopitem_sales_8,prev_shopitem_sales_9,prev_shopitem_sales_10,prev_shopitem_sales_11,prev_shopitem_sales_12,prev_item_sales_1,prev_item_sales_2,prev_item_sales_3,prev_item_sales_4,prev_item_sales_5,prev_item_sales_6,prev_item_sales_7,prev_item_sales_8,prev_item_sales_9,prev_item_sales_10,prev_item_sales_11,prev_item_sales_12,prev_shopitem_price_1,prev_shopitem_price_2,prev_shopitem_price_3,prev_shopitem_price_4,prev_shopitem_price_5,prev_shopitem_price_6,prev_shopitem_price_7,prev_shopitem_price_8,prev_shopitem_price_9,prev_shopitem_price_10,prev_shopitem_price_11,prev_shopitem_price_12,prev_item_price_1,prev_item_price_2,prev_item_price_3,prev_item_price_4,prev_item_price_5,prev_item_price_6,...,_prev_shopitem_sales_19,_prev_shopitem_sales_20,_prev_shopitem_sales_21,_prev_shopitem_sales_22,_prev_shopitem_sales_23,_prev_shopitem_sales_24,_prev_shopitem_sales_25,_prev_shopitem_sales_26,_prev_shopitem_sales_27,_prev_shopitem_sales_28,_prev_shopitem_sales_29,_prev_shopitem_sales_30,_prev_shopitem_sales_31,_prev_shopitem_sales_32,_prev_shopitem_sales_33,last_sale_shop_item,_prev_item_sales_1,_prev_item_sales_2,_prev_item_sales_3,_prev_item_sales_4,_prev_item_sales_5,_prev_item_sales_6,_prev_item_sales_7,_prev_item_sales_8,_prev_item_sales_9,_prev_item_sales_10,_prev_item_sales_11,_prev_item_sales_12,_prev_item_sales_13,_prev_item_sales_14,_prev_item_sales_15,_prev_item_sales_16,_prev_item_sales_17,_prev_item_sales_18,_prev_item_sales_19,_prev_item_sales_20,_prev_item_sales_21,_prev_item_sales_22,_prev_item_sales_23,_prev_item_sales_24,_prev_item_sales_25,_prev_item_sales_26,_prev_item_sales_27,_prev_item_sales_28,_prev_item_sales_29,_prev_item_sales_30,_prev_item_sales_31,_prev_item_sales_32,_prev_item_sales_33,last_sale_item
0,0,0,0,37,1.0,22154,999.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1,1,0,40,2.0,22151,399.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,2,2,0,5,1.0,5603,699.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,3,3,0,5,2.0,5587,199.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,4,4,0,2,1.0,5613,5571.0,59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

Unnamed: 0                   int16
Unnamed: 0.1                 int16
date_block_num               int16
item_category_id             int16
item_cnt_month             float32
item_id                      int16
item_price                 float32
shop_id                      int16
prev_shopitem_sales_1      float32
prev_shopitem_sales_2      float32
prev_shopitem_sales_3      float32
prev_shopitem_sales_4      float32
prev_shopitem_sales_5      float32
prev_shopitem_sales_6      float32
prev_shopitem_sales_7      float32
prev_shopitem_sales_8      float32
prev_shopitem_sales_9      float32
prev_shopitem_sales_10     float32
prev_shopitem_sales_11     float32
prev_shopitem_sales_12     float32
prev_item_sales_1          float32
prev_item_sales_2           object
prev_item_sales_3          float32
prev_item_sales_4          float32
prev_item_sales_5          float32
prev_item_sales_6          float32
prev_item_sales_7          float32
prev_item_sales_8          float32
prev_item_sales_9   

782633

4691435

In [9]:
train_test_set.drop(["Unnamed: 0", "Unnamed: 0.1"], axis = 1, inplace = True)

In [10]:
train_test_set.loc[train_test_set.prev_item_sales_2=="0*0","prev_item_sales_2"] = 0.0
train_test_set.prev_item_sales_2 = train_test_set.prev_item_sales_2.astype("float32")

In [11]:
if kernel_with_output:
    current = time.time()    
    features_added = [col for col in train_test_set if col.startswith("_prev") or col.startswith("prev")]
    baseline_features = ["shop_id", "item_id", "item_category_id", "date_block_num"]+ features_added
    train_test_set["item_cnt_month"] = train_test_set.item_cnt_month.fillna(0).clip(0,20)
    

In [42]:
feature_columns = all_data.drop(["date_block_num", "item_cnt_month"], axis = 1).columns

In [12]:
all_data = train_test_set.copy()

In [13]:
dates = all_data["date_block_num"]
last_block = dates.max() # last_block 34

In [15]:
start_first_level_total = time.perf_counter()
scoringMethod = 'r2'
num_first_level_models = 3 

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [89]:
# Train meta-features M = 15 (12 + 15 = 27)

In [17]:
months_to_generate_meta_features = range(27,last_block +1)
mask = dates.isin(months_to_generate_meta_features)
Target = 'item_cnt_month'

y_all_level2 = all_data[Target][mask].values # choose item_cnt_month where date_block_num from 27..34
X_all_level2 = np.zeros([y_all_level2.shape[0], num_first_level_models])

In [30]:
slice_start = 0

In [43]:
%time
SEED = 0
for cur_block_num in tqdm(months_to_generate_meta_features):
    print('-' * 50)
    print('Start training for month%d'% cur_block_num)
    start_cur_month = time.perf_counter()
    cur_X_train = all_data.loc[dates <  cur_block_num][feature_columns]
    cur_X_test =  all_data.loc[dates == cur_block_num][feature_columns]
    cur_y_train = all_data.loc[dates <  cur_block_num, Target].values
    cur_y_test =  all_data.loc[dates == cur_block_num, Target].values
    # Create Numpy arrays of train, test and target dataframes to feed into models
    train_x = cur_X_train.values
    train_y = cur_y_train.ravel()
    test_x = cur_X_test.values
    test_y = cur_y_test.ravel()
    
    preds = []
    from sklearn.linear_model import (LinearRegression, SGDRegressor)
    import lightgbm as lgb
    sgdr= SGDRegressor(
        penalty = 'l2' ,
        random_state = SEED )
    lgb_params = {
        'feature_fraction': 0.75,
        'metric': 'rmse',
        'nthread':1,
        'min_data_in_leaf': 2**7,
        'bagging_fraction': 0.75,
        'learning_rate': 0.03,
        'objective': 'mse',
        'bagging_seed': 2**7,
        'num_leaves': 2**7,
        'bagging_freq':1,
        'verbose':0}
    estimators = [sgdr]
    for estimator in estimators:
        print('Training Model %d: %s'%(len(preds), estimator.__class__.__name__))
        start = time.perf_counter()
        estimator.fit(train_x, train_y)
        pred_test = estimator.predict(test_x)
        preds.append(pred_test)

        pred_train = estimator.predict(train_x)
        print('Train RMSE for %s is %f' % (estimator.__class__.__name__, sqrt(mean_squared_error(cur_y_train, pred_train))))
        print('Test RMSE for %s is %f' % (estimator.__class__.__name__, sqrt(mean_squared_error(cur_y_test, pred_test))))

        run = time.perf_counter() - start
        print('{} runs for {:.2f} seconds.'.format(estimator.__class__.__name__, run))
        print()
#         import pickle
#         pickle.dump(estimator, open(filename, 'wb'))

    print('Training Model %d: %s'%(len(preds), 'lightgbm'))
    start = time.perf_counter()
    estimator = lgb.train(lgb_params, lgb.Dataset(train_x, label=train_y), 300)
    pred_test = estimator.predict(test_x)
    preds.append(pred_test)

    pred_train = estimator.predict(train_x)
    print('Train RMSE for %s is %f' % ('lightgbm', sqrt(mean_squared_error(cur_y_train, pred_train))))
    print('Test RMSE for %s is %f' % ('lightgbm', sqrt(mean_squared_error(cur_y_test, pred_test))))

    run = time.perf_counter() - start
    print('{} runs for {:.2f} seconds.'.format('lightgbm', run))
    print()

    print('Training Model %d: %s'%(len(preds), 'keras'))
    start = time.perf_counter()
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.wrappers.scikit_learn import KerasRegressor

    def baseline_model():
        # create model
        model = Sequential()
        model.add(Dense(20, input_dim=train_x.shape[1], kernel_initializer='uniform', activation='softplus'))
        model.add(Dense(1, kernel_initializer='uniform', activation = 'relu'))
        # Compile model
        model.compile(loss='mse', optimizer='Nadam', metrics=['mse'])
        # model.compile(loss='mean_squared_error', optimizer='adam')
        return model

    estimator = KerasRegressor(build_fn=baseline_model, verbose=1, epochs=5, batch_size = 55000)
    estimator.fit(train_x, train_y)
    pred_test = estimator.predict(test_x)
    preds.append(pred_test)
    run = time.perf_counter() - start
    print('{} runs for {:.2f} seconds.'.format('lightgbm', run))

    cur_month_run_total = time.perf_counter() - start_cur_month
    print('Total running time was {:.2f} minutes.'.format(cur_month_run_total/60))
    print('-' * 50)

    slice_end = slice_start + cur_X_test.shape[0]
    X_all_level2[ slice_start : slice_end , :] = np.c_[preds].transpose()
    slice_start = slice_end

Wall time: 0 ns


  0%|                                                                                            | 0/8 [00:00<?, ?it/s]

--------------------------------------------------
Start training for month27
Training Model 0: SGDRegressor


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# Split train and test
test_nrow = len(preds[0])
X_train_level2 = X_all_level2[ : -test_nrow, :]
X_test_level2 = X_all_level2[ -test_nrow: , :]
y_train_level2 = y_all_level2[ : -test_nrow]
y_test_level2 = y_all_level2[ -test_nrow : ]
print('%0.2f min: Finish training First level models'%((time.perf_counter() - start_first_level_total)/60))

### 4. Ensembling

In [ ]:
pred_list = {}

In [ ]:
# A. Second level learning model via linear regression
print('Training Second level learning model via linear regression')
from sklearn.linear_model import (LinearRegression, SGDRegressor)
lr = LinearRegression()
lr.fit(X_train_level2, y_train_level2)
# Compute R-squared on the train and test sets.
# print('Train R-squared for %s is %f' %('test_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, lr.predict(X_train_level2)))))
test_preds_lr_stacking = lr.predict(X_test_level2)
train_preds_lr_stacking = lr.predict(X_train_level2)
print('Train R-squared for %s is %f' %('train_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, train_preds_lr_stacking))))
pred_list['test_preds_lr_stacking'] = test_preds_lr_stacking
if Validation:
    print('Test R-squared for %s is %f' %('test_preds_lr_stacking', sqrt(mean_squared_error(y_test_level2, test_preds_lr_stacking))))

In [ ]:
# B. Second level learning model via SGDRegressor
print('Training Second level learning model via SGDRegressor')
sgdr= SGDRegressor(
    penalty = 'l2' ,
    random_state = SEED )
sgdr.fit(X_train_level2, y_train_level2)
# Compute R-squared on the train and test sets.
# print('Train R-squared for %s is %f' %('test_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, lr.predict(X_train_level2)))))
test_preds_sgdr_stacking = sgdr.predict(X_test_level2)
train_preds_sgdr_stacking = sgdr.predict(X_train_level2)
print('Train R-squared for %s is %f' %('train_preds_lr_stacking', sqrt(mean_squared_error(y_train_level2, train_preds_sgdr_stacking))))
pred_list['test_preds_sgdr_stacking'] = test_preds_sgdr_stacking
if Validation:
    print('Test R-squared for %s is %f' %('test_preds_sgdr_stacking', sqrt(mean_squared_error(y_test_level2, test_preds_sgdr_stacking))))
print('%0.2f min: Finish training second level model'%((time.time() - start_time)/60))

In [ ]:
# Submission -------------------------------------------------------------------
if not Validation:
    submission = pd.read_csv('../data/sample_submission.csv')
    ver = 6
    for pred_ver in ['lr_stacking', 'sgdr_stacking']:
        print(pred_list['test_preds_' + pred_ver].clip(0,20).mean())
        submission['item_cnt_month'] = pred_list['test_preds_' + pred_ver].clip(0,20)
        submission[['ID', 'item_cnt_month']].to_csv("../results/09/09.csv", index = False)
print('%0.2f min: Finish running scripts'%((time.time() - start_time)/60))

In [ ]:
!shutdown -s 